In [1]:
import cv2
import mediapipe as mp
import time
import os
class handDetector():
    def __init__(self,mode=False,maxHands=2,detectionCon=0.5,trackCon=0.5,modelComplexity=1):
        self.mode=mode
        self.maxHands=maxHands
        self.modelComplex=modelComplexity
        self.detectionCon=detectionCon
        self.trackCon=trackCon
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.mode,self.maxHands,self.modelComplex,
                                      self.detectionCon,self.trackCon)
        self.mpdraw=mp.solutions.drawing_utils
        self.tipIds=[4,8,12,16,20]
    
    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpdraw.draw_landmarks(img,handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    def findPosition(self,img, handNo=0, draw=True):
        """Lists the position/type of landmarks
        we give in the list and in the list ww have stored
        type and position of the landmarks.
        List has all the lm position"""

        self.lmlist = []

        # check wether any landmark was detected
        if self.results.multi_hand_landmarks:
            #Which hand are we talking about
            myHand = self.results.multi_hand_landmarks[handNo]
            # Get id number and landmark information
            for id, lm in enumerate(myHand.landmark):
                # id will give id of landmark in exact index number
                # height width and channel
                h,w,c = img.shape
                #find the position
                cx,cy = int(lm.x*w), int(lm.y*h) #center
                # print(id,cx,cy)
                self.lmlist.append([id,cx,cy])

                # Draw circle for 0th landmark
                if draw:
                    cv2.circle(img,(cx,cy), 15 , (255,0,255), cv2.FILLED)

        return self.lmlist
    def fingersUp(self):
        fingers=[]
        #thumb
        if self.lmlist[self.tipIds[0]][1]<self.lmlist[self.tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        #4fingers
        for id in range(1,5):
            if self.lmlist[self.tipIds[id]][2]<self.lmlist[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers
        

In [ ]:
import numpy as np
detector=handDetector(detectionCon=0.85)
folderPath='Header'
mylist=os.listdir(folderPath)
overlay=[]
for imPath in mylist:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlay.append(image)
header=overlay[0]
drawColor=(255,0,255)
brushthickness =20
eraserthickness=80
cap=cv2.VideoCapture(0)

cap.set(3,1280)
cap.set(4,720)
xp,yp=0,0
imgcanvas=np.zeros((720,1280,3),np.uint8)
while True:
    success,img=cap.read()
    img=cv2.flip(img,1)
    #1.import image
    #2.find Hand landmarks
    img=detector.findHands(img)
    lmlist=detector.findPosition(img,draw=False)
    if len(lmlist)!=0:
        #print(lmlist)
        #tip of index and middle finger
        x1,y1=lmlist[8][1:]
        x2,y2=lmlist[12][1:]
        #3.checking which finger is up
        fingers=detector.fingersUp()
        #print(fingers)
    #4.if selcton mode if 2fingers are up select
        if fingers[1] and fingers[2]:
            xp,yp=x1,y1
            #checking for the click
            if y1<125:
                if 250<x1<450:
                    header=overlay[0]
                    drawColor=(255,0,255)
                elif 550<x1<750:
                    header=overlay[1]
                    drawColor=(255,0,0)
                elif 755<x1<850:
                    header=overlay[2]
                    drawColor=(0,255,255)
                elif 950<x1<1100:
                    header=overlay[3]
                    drawColor=(0,0,0)
            cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawColor,cv2.FILLED)
        if fingers[1] and fingers[2]==False:
            cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
            if xp==0 and yp==0:
                xp,yp=x1,y1
            if drawColor==(0,0,0):
                cv2.line(img,(xp,yp),(x1,y1),drawColor,eraserthickness)
                cv2.line(imgcanvas,(xp,yp),(x1,y1),drawColor,eraserthickness)
            else:    
                cv2.line(img,(xp,yp),(x1,y1),drawColor,brushthickness)
                cv2.line(imgcanvas,(xp,yp),(x1,y1),drawColor,brushthickness)
            xp,yp=x1,y1
        
    
    imgGray=cv2.cvtColor(imgcanvas,cv2.COLOR_BGR2GRAY)
    _,imagInv=cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
    imginv=cv2.cvtColor(imagInv,cv2.COLOR_GRAY2BGR)

    img=cv2.bitwise_and(img,imginv)
    img=cv2.bitwise_or(img,imgcanvas)
                        #5. if we have drawing mode indexfinger is up
    img[0:125,0:1280]=header
    img=cv2.addWeighted(img,0.5,imgcanvas,0.5,0)
    cv2.imshow('Img',img)
    cv2.imshow('Imgcanvas',imgcanvas)
    cv2.imshow('inv',imginv)
    cv2.waitKey(1)

In [ ]:
overlay[0].shape

In [ ]:
def main():
    pTime=0
    cTime=0
    cap=cv2.VideoCapture(0)
    detector=handDetector()
    while True:
        success,img=cap.read()
        img=detector.findHands(img)
        lmlist=detector.findPosition(img,draw=False)
        if len(lmlist) !=0:
            print(lmlist[2])
        cTime=time.time()
        fps=1/(cTime-pTime)
        pTime=cTime
        cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,
                    3,(255,0,255),3)
        cv2.imshow('Image',img)
        cv2.waitKey(1)

if __name__=="__main__":
    main()